# 報告 2022-11-16
モデル：[2022/10/19版](https://huggingface.co/abctreebank/comparative-NER/commit/4f760c214d16cae4188f060d5051ee4e5ee0bd2b)

## 学習データ／評価データ
9:1になるように，事前に分割した。

In [19]:
import datasets

# NOTE: private repoなので，事前にログインが必要。
ds = datasets.load_dataset(
    "abctreebank/comparative-NER-BCCWJ",
    use_auth_token = True,
)

/home/owner/.cache/pypoetry/virtualenvs/comparative-ner-utils-sv0RmVnD-py3.10/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Using custom data configuration abctreebank--comparative-NER-BCCWJ-c32c3cdce4ba824a
Reusing dataset parquet (/home/owner/.cache/huggingface/datasets/abctreebank___parquet/abctreebank--comparative-NER-BCCWJ-c32c3cdce4ba824a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
ds

DatasetDict({
    test: Dataset({
        features: ['ID', 'tokens', 'comp', 'comments'],
        num_rows: 350
    })
    train: Dataset({
        features: ['ID', 'tokens', 'comp', 'comments'],
        num_rows: 3120
    })
})

* 学習データ数： 3,120文
* テストデータ数： 350文

NOTE: 一語文などを削除したので、数が若干減っている。

### 学習データの例

#### 単文
```yaml
- ID: 5_BCCWJ-ABC-aa-simple
  reference_linear: 妻 が 仕事 に 精出す 一方 、 [[赤沼 は]cont [それ より]prej [もっと]diff [忙しい]deg 。]root
  prediction_linear: '妻 が 仕事 に 精 ##出す 一方 、 [赤 ##沼 は]cont [それ より]prej [もっと]diff
    [忙 ##しい]deg 。'

- ID: 95_bccwj_kurabe_text-aj-simplified
  reference_linear: '[[旧法 が 成立 し た 当時 に 比べ て]prej 、 私 たち の 食生活 は [格段 に]diff [豊か]deg
    に なっ た 。]root'
  prediction_linear: '[旧 ##法 が 成立 し た 当時 に 比べ て]prej 、 私 たち の 食 ##生活 は [格段 に]diff
    [豊か]deg に なっ た 。'

- ID: 35_bccwj_kurabe_text-ah-simplified
  reference_linear: '[[アカ ナマコ の 成長 は]cont [アオナマコ に]prej [比べ]prej [若干]diff [劣っ]deg
    て い た 。]root'
  prediction_linear: '[アカ ナ ##マコ]cont の 成長 は [アオ ##ナ ##マコ に 比べ]prej [若干]diff
    [劣っ]deg て い た 。'
  errors:
  - - cont
    - WRONG_SPAN
```

#### 連用節
```yaml
- ID: 23_bccwj_kurabe_text-af-simplified
  reference_linear: ところが 、 一 九 八 五 年 九月 の プラザ 合意 以降 、 [[円 が]cont [ドル に 比べ て]prej [百
    ％ 以上]diff [はね上がり]root]deg 、 突然 日本 は アメリカ より はるか に 高 コスト の 国 に なり まし た 。
  prediction_linear: 'ところが 、 一 九 八 五 年 九 ##月 の プラザ 合意 以降 、 [円 が]cont [ドル に 比べ
    て]prej [百 % 以上]diff [はね ##上がり]deg 、 突然 [日本 は]cont [アメリカ より]prej [はるか]diff に [高
    コスト]deg の 国 に なり まし た 。'
  errors:
  - - cont
    - SPURIOUS
  - - prej
    - SPURIOUS
  - - diff
    - SPURIOUS
  - - deg
    - SPURIOUS
```

#### 連体節
```yaml
- ID: 21_BCCWJ-ABC-as-simple
  reference_linear: 三 十 歳 の サラリーマン が [自分 より]prej [七]diff [、]diff [[[八 歳]diff [年下]root]deg]cont
    の 「 新入 社員 の 気持ち が わから ない 」 と 言っ て いる 。
  prediction_linear: '[三 十 歳 の サラリーマン が]cont [自分 より]prej [七 、 八 歳]diff [年下]deg
    の 「 新入 社員 の 気持ち が わから ない 」 と 言っ て いる 。'
  errors:
  - - cont
    - SPURIOUS
  - - diff
    - WRONG_SPAN
  - - cont
    - MISSING
  comments:
  - これは難しい。鍵カッコがあるので、そもそも構造を取るのが難しい。
 
- ID: 99_BCCWJ-ABC-au-simple
  reference_linear: 何 畳 ある か わから ない くらい 、 [[教室 より も]prej [広い]root]deg [部屋]cont 。
  prediction_linear: '何 畳 ある か わから ない くらい 、 [教室 より も]prej [広い]deg [部屋]cont 。
   '
```

## モデルの設定
* 使用した事前学習モデル： https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking
* このモデルの上で，NER（固有表現認識）の一種として，与えられた文のどのspanが，比較構文の要素に相当するのかについてのモデルを構築。
* 比較構文の要素：
    * prej(acent)：「より」句
    * cont(rast)：比較対象
    * diff(erence)：差の表現
    * deg(ree)：程度表現
    * root：比較構文の最大スコープ（NERモデルにおいては取り除いた）
* 例： [ [太郎が]cont [花子よりも]prej [3cm]diff [高い]deg ]root ことは意外だった。

In [21]:
# 学習パラメータ

training_args = dict(
    # output_dir = str(output_path),

    # エポック数
    num_train_epochs = 27,

    # バッチのサイズ
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 128,

    # 学習率
    learning_rate = 5e-5,
    
    warmup_steps = 200,
    weight_decay = 0,
    # save_strategy = IntervalStrategy.STEPS,
    save_steps = 1000,
    do_eval = True,
    # evaluation_strategy = IntervalStrategy.STEPS,
    eval_steps = 109,
    include_inputs_for_metrics = True,

    # 乱数シード
    seed = 2630987289,

    # logging_dir = str(output_path / "logs"),
    logging_steps= 10,
)

## 学習結果

In [22]:
result = {"score_spanwise_details": {"cont": {"CORRECT": 107, "WRONG_SPAN": 73, "SPURIOUS": 67, "MISSING": 53, "WRONG_LABEL": 1, "WRONG_LABEL_SPAN": 9}, "prej": {"CORRECT": 229, "WRONG_LABEL_SPAN": 6, "WRONG_SPAN": 33, "MISSING": 5, "SPURIOUS": 32}, "diff": {"CORRECT": 63, "MISSING": 10, "SPURIOUS": 14, "WRONG_SPAN": 10, "WRONG_LABEL_SPAN": 1}, "deg": {"CORRECT": 218, "WRONG_SPAN": 15, "WRONG_LABEL": 3, "SPURIOUS": 60, "MISSING": 34, "WRONG_LABEL_SPAN": 4}}, "score_spanwise": {"cont": {"possible_entries": 243, "actual_entries": 257, "precision_strict": 0.4163424124513619, "recall_strict": 0.4403292181069959, "F1_strict": 0.574, "precision_partial": 0.5583657587548638, "recall_partial": 0.5905349794238683}, "prej": {"possible_entries": 273, "actual_entries": 300, "precision_strict": 0.7633333333333333, "recall_strict": 0.8388278388278388, "F1_strict": 0.8568935427574172, "precision_partial": 0.8183333333333334, "recall_partial": 0.8992673992673993}, "diff": {"possible_entries": 84, "actual_entries": 88, "precision_strict": 0.7159090909090909, "recall_strict": 0.75, "F1_strict": 0.7906976744186046, "precision_partial": 0.7727272727272727, "recall_partial": 0.8095238095238095}, "deg": {"possible_entries": 274, "actual_entries": 300, "precision_strict": 0.7266666666666667, "recall_strict": 0.7956204379562044, "F1_strict": 0.7857142857142858, "precision_partial": 0.7516666666666667, "recall_partial": 0.822992700729927}}, "score_spanwise_F1_strict": 0.7518263757225769, "score_tokenwise": {"IGNORE": {"precision": 0.0, "recall": 0.0, "f1-score": 0.0, "support": 0}, "O": {"precision": 0.9349922839506173, "recall": 0.9109190001879346, "f1-score": 0.9227986673012851, "support": 5321}, "B-deg": {"precision": 0.761437908496732, "recall": 0.8411552346570397, "f1-score": 0.7993138936535162, "support": 277}, "B-prej": {"precision": 0.8160535117056856, "recall": 0.8591549295774648, "f1-score": 0.83704974271012, "support": 284}, "B-cont": {"precision": 0.6639004149377593, "recall": 0.6986899563318777, "f1-score": 0.6808510638297872, "support": 229}, "B-diff": {"precision": 0.7419354838709677, "recall": 0.7752808988764045, "f1-score": 0.7582417582417582, "support": 89}, "I-deg": {"precision": 0.6923076923076923, "recall": 0.7397260273972602, "f1-score": 0.7152317880794701, "support": 73}, "I-prej": {"precision": 0.8496, "recall": 0.959349593495935, "f1-score": 0.9011455239711498, "support": 1107}, "I-cont": {"precision": 0.7381703470031545, "recall": 0.6733812949640288, "f1-score": 0.7042889390519188, "support": 695}, "I-diff": {"precision": 0.7654320987654321, "recall": 0.6813186813186813, "f1-score": 0.7209302325581395, "support": 91}, "micro avg": {"precision": 0.881582169973059, "recall": 0.881582169973059, "f1-score": 0.881582169973059, "support": 8166}, "macro avg": {"precision": 0.6963829741038041, "recall": 0.7138975616806627, "f1-score": 0.7039851609397145, "support": 8166}, "weighted avg": {"precision": 0.8828759818341678, "recall": 0.881582169973059, "f1-score": 0.8814116671139006, "support": 8166}}}

In [71]:
import pandas as pd

# データカウント
df_res_count = pd.DataFrame.from_dict(
    result["score_spanwise_details"],
    orient = "index",
).fillna(0)

df_res_count

,CORRECT,WRONG_SPAN,SPURIOUS,MISSING,WRONG_LABEL,WRONG_LABEL_SPAN
cont,107,73,67,53,1.0,9
prej,229,33,32,5,0.0,6
diff,63,10,14,10,0.0,1
deg,218,15,60,34,3.0,4


* CORRECT: ぴったり
* WRONG_SPAN: spanにずれがある。
* SPURIOUS: 正解データにないspanを予測してしまっている。
* MISSING: 正解データにあるspanを予測できていない。
* WRONG_LABEL_SPAN: spanにずれがあり，かつ，ラベルも間違っている。
* WRONG_LABEL: spanにずれはないが，ラベルが間違っている。

In [78]:
# metricsの計算でバグあり、こちらに差し替え

def calc_corr(row: pd.Series):
    res = {
        "possible_entries": row["CORRECT"] + row["WRONG_SPAN"] + row["WRONG_LABEL"] + row["WRONG_LABEL_SPAN"] + row["MISSING"]
    }
    res["actual_entires"] = (
        res["possible_entries"] - row["MISSING"] + row["SPURIOUS"]
    )
    res["precision_strict"] = (
        row["CORRECT"] / res["actual_entires"]
    )
    res["recall_strict"] = (
        row["CORRECT"] / res["possible_entries"]
    )
    res["F1_strict"] = (
        2 * res["recall_strict"] * res["precision_strict"]
        / (res["recall_strict"] + res["precision_strict"])
    )
    res["precision_partial"] = (
        (row["CORRECT"] + 0.5 * row["WRONG_SPAN"])
        / res["actual_entires"]
    )
    res["recall_partial"] = (
        (row["CORRECT"] + 0.5 * row["WRONG_SPAN"])
        / res["possible_entries"]
    )
    res["F1_partial"] = (
        2 * res["recall_partial"] * res["precision_partial"]
        / (res["recall_partial"] + res["precision_partial"])
    )

    return res

In [79]:
# 統計
df_res_stat = pd.DataFrame.from_records(
    df_res_count.apply(
        calc_corr,
        axis = 1
    ),
    index = df_res_count.index
)

df_res_stat

,possible_entries,actual_entires,precision_strict,recall_strict,F1_strict,precision_partial,recall_partial,F1_partial
cont,243.0,257.0,0.416342,0.440329,0.428000,0.558366,0.590535,0.574000
prej,273.0,300.0,0.763333,0.838828,0.799302,0.818333,0.899267,0.856894
diff,84.0,88.0,0.715909,0.750000,0.732558,0.772727,0.809524,0.790698
deg,274.0,300.0,0.726667,0.795620,0.759582,0.751667,0.822993,0.785714


### 凡例

* `possible_entries`: 予測されたspanの数
    ```
    ct["CORRECT"] + ct["WRONG_SPAN"] + ct["WRONG_LABEL"] + ct["WRONG_LABEL_SPAN"] + ct["MISSING"]
    ```
* `actual_entires`: テストデータにあるspanの数
    ```
    res["possible_entries"] - ct["MISSING"] + ct["SPURIOUS"]
    ```
* `precision_strict`: 予測のうち，当たっているものの数
    ```
    ct["CORRECT"] / res["actual_entries"]
    ```
* `recall_strict`: テストデータにあるもののうち，予測されたspanの数
    ```
    ct["CORRECT"] / res["possible_entries"]
    ```
* `precision_partial`：strictよりも緩い。 WRONG_SPANを50%カウントに入れている。
    ```
    (ct["CORRECT"] + 0.5 * ct["WRONG_SPAN"]) / res["actual_entries"]
    ```
* `recall_partial`
    ```
    (ct["CORRECT"] + 0.5 * ct["WRONG_SPAN"]) / res["possible_entries"]
    ```
* F1はprecisionとrecallの調和平均

In [80]:
# F1_strictの単純平均。ラベルのカウントで重みづけることはしていない。
df_res_stat["F1_strict"].mean()

0.6798604851971882

In [82]:
# F1_partialの単純平均
df_res_stat["F1_partial"].mean()

0.7518263757225769

## 目視コメント

In [91]:
import ruamel.yaml
yaml = ruamel.yaml.YAML()

# 目視でのコメントを読み込む
with open("../../comp-proto/NER-test-result_2022-10/NER-test-result-dump_2022-10-19.yaml") as f_dump:
    data_dump = yaml.load(f_dump)

df_dump = pd.DataFrame(data_dump).set_index("ID")

修正の見込みがありそうな（目視で判断）間違いのデータを、修正できたとしたときの指標を計算する。

In [88]:
import itertools
from collections import Counter

reeval = df_dump["custom_contrast_reevaluation"]
df_dump_include = reeval.map(
    lambda r: r not in ("correct", "almost correct", "next time")
) & reeval.notna()

counter_cont_hard = Counter(
    val
    for feat, val in itertools.chain.from_iterable(
        df_dump[df_dump_include]["errors"]
    )
    if feat == "cont"
)

# CORRECTになる予定のデータ数
counter_cont_hard

Counter({'MISSING': 22, 'SPURIOUS': 53, 'WRONG_SPAN': 27, 'WRONG_LABEL': 1})

In [89]:
df_res_count_modified = df_res_count.copy().fillna(0)

for k, v in counter_cont_hard.items():
    df_res_count_modified.loc["cont", k] -= v

df_res_count_modified.loc["cont", "CORRECT"] += sum(counter_cont_hard.values())

# not difficult contrast errorを正解とみなしたときのカウント
df_res_count_modified

,CORRECT,WRONG_SPAN,SPURIOUS,MISSING,WRONG_LABEL,WRONG_LABEL_SPAN
cont,210,46,14,31,0.0,9
prej,229,33,32,5,0.0,6
diff,63,10,14,10,0.0,1
deg,218,15,60,34,3.0,4


In [90]:
df_res_stat_modified = pd.DataFrame.from_records(
    df_res_count_modified.apply(calc_corr, axis = 1),
    index = df_res_count_modified.index
)

# not difficult contrast errorを正解とみなしたときの指標
df_res_stat_modified

,possible_entries,actual_entires,precision_strict,recall_strict,F1_strict,precision_partial,recall_partial,F1_partial
cont,296.0,279.0,0.752688,0.709459,0.730435,0.835125,0.787162,0.810435
prej,273.0,300.0,0.763333,0.838828,0.799302,0.818333,0.899267,0.856894
diff,84.0,88.0,0.715909,0.750000,0.732558,0.772727,0.809524,0.790698
deg,274.0,300.0,0.726667,0.795620,0.759582,0.751667,0.822993,0.785714


In [85]:
# F1_strictの単純平均
df_res_stat_modified["F1_strict"].mean()

0.7554691808493621

In [86]:
# F1_partialの単純平均
df_res_stat_modified["F1_partial"].mean()

0.8109350713747507